<a href="https://colab.research.google.com/github/Coperr/amd-stock-ml-project/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
import platform
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [33]:
from google.colab import files
uploaded = files.upload()

Saving AMD-Stock-Price-History.csv to AMD-Stock-Price-History (3).csv


In [34]:
data = pd.read_csv("/content/AMD-Stock-Price-History.csv", low_memory=False)

In [46]:
data

,Date,Price,Open,High,Low,Vol.,Change %
2225,2016-01-28,2.08,2.16,2.17,2.07,7120000.0,-0.0235
2224,2016-01-29,2.20,2.09,2.20,2.07,12000000.0,0.0577
2223,2016-02-01,2.14,2.17,2.19,2.11,8820000.0,-0.0273
2222,2016-02-02,1.99,2.11,2.14,1.96,11220000.0,-0.0701
2221,2016-02-03,2.07,2.04,2.08,1.95,9980000.0,0.0402
...,...,...,...,...,...,...,...
4,2024-11-21,137.49,138.87,140.28,134.93,29310000.0,-0.0008
3,2024-11-22,138.35,137.35,139.13,137.04,21780000.0,0.0063
2,2024-11-25,141.13,140.49,142.35,139.05,30920000.0,0.0201
1,2024-11-26,137.72,142.55,142.80,136.62,32090000.0,-0.0242


In [56]:
def convert_volume(volume_str):
    if isinstance(volume_str, str):
        if 'M' in volume_str:
            return float(volume_str.replace('M', '').replace(',', '').strip()) * 1_000_000
        elif 'B' in volume_str:
            return float(volume_str.replace('B', '').replace(',', '').strip()) * 1_000_000_000
        else:
            return float(volume_str.replace(',', '').strip())
    return volume_str

def convert_change_percentage(data):
    if 'Change %' not in data.columns:
        raise ValueError("DataFrame must contain a 'Change %' column.")

    if data['Change %'].dtype == object:
        data['Change %'] = data['Change %'].str.replace('%', '').astype(float) / 100
    elif data['Change %'].max() > 1 or data['Change %'].min() < -1:
        data['Change %'] = data['Change %'] / 100


data['Date'] = pd.to_datetime(data['Date'])

data['Vol.'] = data['Vol.'].apply(convert_volume)

convert_change_percentage(data)

data.sort_values('Date', inplace=True)

features = data[['Open', 'High', 'Low', 'Vol.', 'Change %']]
target = data['Price']

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared Score: {r2}")

latest_features = features.iloc[-1:]
predicted_price = pipeline.predict(latest_features)
print(f"Predicted Next Stock Price: {predicted_price[0]}")

Mean Squared Error: 0.8420173241283349
R-squared Score: 0.9996778221721183
Predicted Next Stock Price: 134.63708692860314
